# Project 1 — Berry–Esseen Rate for Fixed Degree $d$

Empirical confirmation that the overlap  
$$X_N := \sqrt{N}\langle q, u_2\rangle$$
between a fixed test vector $q\perp\mathbf{1}$ and the second eigenvector $u_2$ of a **random $d$-regular graph** converges to $\mathcal{N}(0, 1)$ at the **optimal Berry–Esseen rate**
$$\sup_x |\mathbb{P}(X_N\le x)-\Phi(x)| = \Theta(N^{-1/6}),$$
as established in Nagel (2025) and Huang–Yau (2023).

**Credits**: This notebook was written by [Hershraj Niranjani](https://hershrajn.com)

---

## Experimental Design

| Stage | What we do | Key parameters |
|-------|------------|----------------|
| **Graph generation** | Build $M \approx 7,000$ simple $d$-regular graphs for every $(N, d)$ using edge-swapping Monte Carlo. | $d \in \{3,5,8\}$; $N \in \{1,000, 10,000, 100,000, 500,000, 1,000,000\}$; Rust backend with $3 \times N \times d$ edge swaps per graph. |
| **Storage format** | Save as compressed sparse CSR matrices (`.npz` files) for memory efficiency. | `scipy.sparse.save_npz` with compression; adjacency matrices stored as uint8. |
| **Spectral step** | Form the normalised adjacency $\tilde{A} = A / \sqrt{d-1}$ and extract $u_2$. | `scipy.sparse.linalg.eigsh` with `k=2`, `which="LA"`, `tol=1e-2` |
| **Statistic** | Compute $X_N$ with $q = e_1 - \frac{1}{N}\mathbf{1}$ (normalised). | Any deterministic $q \perp \mathbf{1}$ works. |
| **Distance metric** | Kolmogorov–Smirnov distance $D_N = \| F_N(x) - \Phi(x)\|$. | Use SciPy's `stats.kstest` with `norm.cdf` reference. |
| **Rate extraction** | Linear regression of $\log D_N$ on $\log N$. | Slope $\approx -1/6$ corroborates theory. |


In [ ]:
# Note: This was ran in a python virtual environment with python version 3.13.5
# Install the necessary dependencies
%pip install -r requirements.txt

In [ ]:
from pathlib import Path
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os 
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import re
import scipy.sparse.linalg as spla
import scipy.sparse as sp
from scipy.stats import kstest, norm, gaussian_kde
import pandas as pd
from typing import List, Sequence

RANDOM_SEED = 42 # 42 for reproducibility

BASE = Path(os.path.abspath(''))

FIGURES = BASE / "figures"
FIGURES.mkdir(exist_ok=True) # Ensure that the figures directory exists

DATA = BASE / "data"
DATA.mkdir(exist_ok=True) # Ensure that the data directory exists

# First, generate graphs if they don't exist
if not any(DATA.glob("d*/g_*.npz")):
    print("No graph files found. Running graph generation...")
    exec(open("generate_graphs.py").read())
    print("Graph generation complete.")
else:
    print("Found existing graph files.")

NUM_GRAPHS_TO_LOAD = 2_000 # Number of graphs to load in per (n, d) configuration

print("Setup complete.")

In [ ]:
def load_graphs(file_names: Sequence[str], in_dir: str | Path) -> List[sp.csr_matrix]:
    """Load .npz CSR matrices (saved by generate_graphs.py) into sparse matrices."""
    graphs: List[sp.csr_matrix] = []
    for file_name in file_names:
        file_path = Path(in_dir) / file_name
        A = sp.load_npz(file_path)
        graphs.append(A)
    return graphs

## Run the analysis
Make sure that you have generated the graphs using the `generate_graphs.py` script

In [ ]:
BATCH_SIZE         = 1024
N_JOBS             = max(cpu_count() - 1, 1)

def _x_stat_from_file(path: Path, q: np.ndarray) -> float:
    try:
        # Load CSR matrix directly (new format from generate_graphs.py)
        A: sp.csr_matrix = sp.load_npz(path)
        n = A.shape[0]
        
        # Verify this is actually a valid adjacency matrix
        if A.shape[0] != A.shape[1]:
            tqdm.write(f"Error: {path} is not square matrix")
            return None
            
    except Exception as e:
        tqdm.write(f"Error loading {path}: {e}")
        return None

    try:
        # Convert to float32 for computation
        A = A.astype(np.float32, copy=False)

        # Get degree from first row (regular graph) - count nonzero entries
        d = A.getrow(0).nnz
        if d == 0:
            tqdm.write(f"Error: {path} has zero degree")
            return None
            
        # Normalize by sqrt(d-1) for spectral analysis
        A.data *= 1.0 / np.sqrt(np.float32(max(1, d - 1)))

        # Cast to float64 for eigsh
        A_float64 = A.astype(np.float64, copy=False)
        vals, vecs = spla.eigsh(A_float64, k=2, which="LA", tol=1e-2, maxiter=1000)
        
        # Validate eigenvalues and eigenvectors
        if not np.isfinite(vals).all():
            tqdm.write(f"Error: {path} has non-finite eigenvalues")
            return None
            
        if not np.isfinite(vecs).all():
            tqdm.write(f"Error: {path} has non-finite eigenvectors")
            return None
        
        # Get second largest eigenvalue's eigenvector
        u2 = vecs[:, 0] if vals[0] < vals[1] else vecs[:, 1]
        
        # Ensure u2 is properly normalized and finite
        if not np.isfinite(u2).all():
            tqdm.write(f"Error: {path} has non-finite u2")
            return None
            
        u2_norm = np.linalg.norm(u2)
        if u2_norm == 0 or not np.isfinite(u2_norm):
            tqdm.write(f"Error: {path} has zero or invalid u2 norm")
            return None
            
        u2 = u2 / u2_norm  # Ensure unit normalization
        
        # Compute the overlap with bounds checking
        overlap = np.dot(q, u2)
        if not np.isfinite(overlap):
            tqdm.write(f"Error: {path} has non-finite overlap")
            return None
            
        result = np.sqrt(n) * float(overlap)
        
        # Final bounds check
        if not np.isfinite(result):
            tqdm.write(f"Error: {path} has non-finite result")
            return None
            
        return result
        
    except Exception as e:
        tqdm.write(f"Error computing eigenvector for {path}: {e}")
        return None

# Scan for available graph files with corrected pattern matching
graph_files = []
for d_folder in DATA.glob("d*"):
    if not d_folder.is_dir():
        continue
    graph_files.extend(d_folder.glob("g_*.npz"))

if not graph_files:
    print("No graph files found! Please run generate_graphs.py first.")
    exit()

print(f"Found {len(graph_files)} graph files")

# Parse filenames: g_d{d}_n{n}_{seed}.npz
pat = re.compile(r"g_d(\d+)_n(\d+)_(\d+)\.npz")
groups: dict[tuple[int, int], list[Path]] = {}

for p in graph_files:
    match = pat.match(p.name)
    if not match:
        tqdm.write(f"Warning: Skipping file with unexpected name format: {p.name}")
        continue
    
    d, n, seed = map(int, match.groups())
    groups.setdefault((n, d), []).append(p)

print(f"Found graphs for {len(groups)} (n,d) combinations")

rows = []
for (n, d), paths in sorted(groups.items()):
    if len(paths) == 0:
        continue
        
    paths = paths[:NUM_GRAPHS_TO_LOAD]              # deterministic slice
    q     = np.append([1.0], np.zeros(n - 1)) - 1.0 / n
    q    /= np.linalg.norm(q)

    X_vals = []

    if n > 50_000:
        current_batch_size = 128 # reduce memory usage for large n
    else:
        current_batch_size = BATCH_SIZE

    for i in range(0, len(paths), current_batch_size):
        batch = paths[i : i + current_batch_size]
        X_batch = Parallel(
            n_jobs=N_JOBS,
            backend="loky",
            pre_dispatch="2*n_jobs"                 # keeps memory bounded
        )(delayed(_x_stat_from_file)(p, q) for p in batch)
        
        # Filter out None results
        X_vals.extend([x for x in X_batch if x is not None])

        tqdm.write(f"[{n=}, {d=}] processed {i + len(batch):,}/{len(paths):,}")
    
    if len(X_vals) < 10:  # Need minimum number of samples
        tqdm.write(f"Warning: Only got {len(X_vals)} valid samples for n={n}, d={d}")
        continue
        
    D_n, _ = kstest(X_vals, norm.cdf)
    var_n    = float(np.var(X_vals, ddof=1))            # unbiased sample variance
    hist_y, hist_x = np.histogram(
        X_vals, bins="auto", density=True
    ) 
    kde_x = np.linspace(min(X_vals), max(X_vals), 200)
    kde_y = gaussian_kde(X_vals)(kde_x)
    rows.append(
        dict(
            n=n,
            d=d,
            D_n=D_n,
            var_n=var_n,
            n_graphs=len(X_vals),
            hist_x=hist_x,           # bin edges  (len = k+1)
            hist_y=hist_y,           # densities  (len = k)
            kde_x=kde_x,           # uncomment if you keep KDE
            kde_y=kde_y,
        )
    )
    tqdm.write(
        f"[n={n:,}, d={d}] D_n={D_n:.4f}, Var={var_n:.3f} from {len(X_vals):,} graphs"
    )

print("All batches done.")
full_df = pd.DataFrame(rows)
if len(full_df) > 0:
    print(f"Successfully processed {len(full_df)} (n,d) configurations")
    display(full_df.head(50))
else:
    print("No data processed successfully. Check graph files and try again.")

In [ ]:
if len(full_df) == 0:
    print("No data to analyze. Please check graph generation and file paths.")
else:
    limited_df = full_df[ full_df["d"] < full_df["n"] ** 0.25 ] # Ensure d(N) < N^0.25
    res_df = limited_df.copy()
    print(f"Filtered to {len(res_df)} configurations satisfying d < N^0.25")

In [ ]:


# Visualize the Berry-Esseen rate for fixed d (KS distance)
fig, ax = plt.subplots(figsize=(6, 4))

for d, sub in res_df.groupby("d"):
    sub = sub.sort_values("n")

    xs = np.log10(sub["n"].values)
    ys = np.log10(sub["D_n"].clip(lower=1e-12).values)  # avoid log10(0)

    # least‑squares slope on log‑log scale
    m, b = np.polyfit(xs, ys, 1)
    ax.plot(xs, ys, "o-", label=fr"$d={d}$  slope={m:.3f}")

# reference line with slope −1/6
N_min, N_max = res_df["n"].min(), res_df["n"].max()
ref_x = np.array([N_min, N_max])
y0 = np.log10(res_df["D_n"].max())          # anchor at left edge
ref_y = y0 - (1/6) * (np.log10(ref_x) - np.log10(N_min))
ax.plot(np.log10(ref_x), ref_y, "--", color="gray", lw=1.2,
        label=r"reference slope $-1/6$")

ax.set_xlabel(r"$\log_{10} N$")
ax.set_ylabel(r"$\log_{10} D_N$")
ax.set_title("Berry-Esseen rate for fixed $d$ (KS distance)")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the Variance
fig, ax = plt.subplots(figsize=(6, 4))

for d, sub in res_df.groupby("d"):
    sub = sub.sort_values("n")
    ax.plot(
        np.log10(sub["n"]),
        sub["var_n"],
        "o-",
        label=f"d={d}",
    )

ax.axhline(1.0, ls="--", color="gray", lw=1.2, label="theory σ² = 1")
ax.set_xlabel(r"$\log_{10} N$")
ax.set_ylabel(r"sample variance  $\operatorname{Var}[X_N]$")
ax.set_title("Variance of $X_N$ for fixed $d$")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the empirical density of X_N vs Standard Normal
fig, ax = plt.subplots(figsize=(6, 4))

for d in sorted(res_df["d"].unique()):
    row = res_df[res_df["d"] == d].sort_values("n").iloc[-1]  # largest N for this d
    ax.plot(
        row["kde_x"], row["kde_y"],
        label=f"d={d}, N={row['n']:,}"
    )

# standard normal pdf for reference
x_grid = np.linspace(-4, 4, 400)
ax.plot(x_grid, norm.pdf(x_grid), "k--", lw=1.5, label="N(0, 1)")

ax.set_xlabel(r"$x$")
ax.set_ylabel("density")
ax.set_title("Empirical density of $X_N$ vs Standard Normal")
ax.legend(fontsize=8)
plt.tight_layout()
plt.show()

# Project 2 — Berry–Esseen Rate in the Sparse-Growing-Degree Regime

We now let the degree grow slowly with graph size,  
$$d = d(N) \le N^{1/4},$$
and test the refined prediction (Nagel 2025; Huang–Yau 2023) that the **rescaled KS error**

$$\tilde{D}_N = \frac{D_N}{\sqrt{d}}$$

still obeys the optimal rate  

$$\tilde{D}_N = \Theta(N^{-1/6}).$$

Equivalently,

$$\sup_x \left|\mathbb{P}(X_N \le x) - \Phi(x)\right| = \Theta(\sqrt{d}N^{-1/6}),$$

where $X_N = \sqrt{N}\langle q, u_2\rangle$.

---

## Experimental Design

| Stage | What we do | Key parameters |
|-------|------------|----------------|
| **Graph generation** | Build simple $d(N)$-regular graphs using the same edge-swapping Monte Carlo method as Project 1. | $d \in \{3,5,8,10\}$ across $N \in \{1,000, 10,000, 50,000\}$; analyze $(N,d)$ pairs satisfying $d \leq N^{0.25}$; $\approx$ 5k graphs per configuration. |
| **Backend optimization** | Leverage Rust backend for fast batch generation with adaptive batch sizes based on $N$. | Batch size: 512 for $N \leq 10k$, 256 for $N \leq 50k$, 64 for $N > 50k$ to manage memory. |
| **Storage format** | Same compressed sparse CSR format as Project 1 for consistent memory usage. | `scipy.sparse.save_npz` with uint8 adjacency data and int32/int64 indices based on $N$. |
| **Spectral step** | Form the normalised adjacency $\tilde{A} = A / \sqrt{d-1}$ and extract $u_2$. | Same `scipy.sparse.linalg.eigsh` configuration as Project 1. |
| **Statistic** | $X_N = \sqrt{N}\langle q, u_2\rangle$ with $q = e_1 - \frac{1}{N}\mathbf{1}$ (normalised). | Same deterministic $q \perp \mathbf{1}$ as in Project 1. |
| **Distance metric** | $\tilde{D}_N = D_N / \sqrt{d}$ where $D_N$ is the KS distance between $X_N$ and $\mathcal{N}(0,1)$. | `scipy.stats.kstest` with rescaling by $\sqrt{d}$ |
| **Rate extraction** | Regress $\log_{10}\tilde{D}_N$ on $\log_{10}N$. | Slope $\approx -1/6$ confirms universality after the $\sqrt{d}$ factor. |


In [ ]:
# Visualize the scaled KS distance for growing degree d(N) ≤ N^0.25
fig, ax = plt.subplots(figsize=(6, 4))

# points: all (n,d) pairs, colour‑by‑d for readability
scatter = ax.scatter(
    np.log10(limited_df["n"]),
    np.log10(limited_df["D_n"] / np.sqrt(limited_df["d"])),
    c=limited_df["d"], cmap="viridis", s=35
)

# global least‑squares slope in log–log space
xs = np.log10(limited_df["n"].values)
ys = np.log10(limited_df["D_n"].values / np.sqrt(limited_df["d"].values))
m, b = np.polyfit(xs, ys, 1)
ax.plot(xs, m * xs + b, "k--", lw=1.5, label=f"fit slope = {m:.3f}")

# reference – theoretical slope −1/6
N_min, N_max = limited_df["n"].min(), limited_df["n"].max()
ref_x = np.array([N_min, N_max])
ref_y = b - (1 / 6) * (np.log10(ref_x) - np.log10(N_min))
ax.plot(np.log10(ref_x), ref_y, "r:", lw=1.2, label=f"expected slope {(-1 / 6):.3f}")

ax.set_xlabel("log10 N")
ax.set_ylabel("log10 [D_N / sqrt(d)]")
ax.set_title("Scaled KS distance for growing degree d(N) ≤ N^0.25")
ax.legend()
cbar = plt.colorbar(scatter, ax=ax, label="degree d")
plt.tight_layout()
plt.show()

In [ ]:
# See how this looks for points not covered by the bound d(N) ≤ N^0.25
# ── Points with d(N) > N^0.25  ──────────────────────────────────────────────
outside_df = full_df[full_df["d"] > full_df["n"] ** 0.25]

if len(outside_df) == 0:
    print("No points outside the bound found.")
else:
    fig, ax = plt.subplots(figsize=(6, 4))

    # scatter: log‑log of scaled KS distance, colour by degree
    scatter = ax.scatter(
        np.log10(outside_df["n"]),
        np.log10(outside_df["D_n"] / np.sqrt(outside_df["d"])),
        c=outside_df["d"],
        cmap="plasma",
        s=35,
        marker="o",
        label="outside bound"
    )

    # least‑squares slope for these out‑of‑bound points
    xs = np.log10(outside_df["n"].values)
    ys = np.log10(outside_df["D_n"].values / np.sqrt(outside_df["d"].values))
    m, b = np.polyfit(xs, ys, 1)
    ax.plot(xs, m * xs + b, "k--", lw=1.5, label=f"fit slope = {m:.3f}")

    # theoretical reference: slope −1/6 anchored at smallest N
    N_min, N_max = outƒside_df["n"].min(), outside_df["n"].max()
    ref_x = np.array([N_min, N_max])
    ref_y = b - (1 / 6) * (np.log10(ref_x) - np.log10(N_min))
    ax.plot(np.log10(ref_x), ref_y, "r:", lw=1.2, label="expected slope -1/6")

    ax.set_xlabel("log10 N")
    ax.set_ylabel("log10 [D_N / sqrt(d)]")
    ax.set_title("Scaled KS distance for d(N) > N^0.25 (outside bound)")
    ax.legend()
    plt.colorbar(scatter, ax=ax, label="degree d")
    plt.tight_layout()
    plt.show()
